Modified: Jul 20, 2019

# Sign Distance Function & Gradient field visualization
- Visualize various sdfs and its gradient fields
- Enable to explore the parameter spaces of the sdfs using `params`



In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
from scipy.signal import correlate2d
import pandas as pd
    
from pathlib import Path
from pprint import pprint as pp

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts, dim
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams

import panel as pn


hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")


In [ ]:
from utils import get_mro as mro, nprint
import utils as u

from sdfs import *

### Set visualization options

In [ ]:
%opts Image [colorbar=True, active_tools=['wheel_zoom'], tools=['hover']] Curve [tools=['hover'], active_tools=['wheel_zoom']] RGB [active_tools=['wheel_zoom'], tools=['hover']]

In [ ]:
H, W = 300,300
img_opts = opts.Image(height=H, width=W, colorbar_position='bottom')
vfield_opts = opts.VectorField(width=W, height=H, color='Magnitude',
#                                magnitude=dim('Magnitude').norm()*0.2,
                               pivot='tip',
                               rescale_lengths=True)
curve_opts = opts.Points(size=5,width=W, height=H, padding=0.1, 
#                             xlim=(-10,10), ylim=(-10,10),
#                         color=dim('p')*256-50
                        )
contour_opts = opts.Contours(width=W, height=H, 
                             colorbar=False, 
                             tools=['hover'])

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

### SDF evaluation wrapper
- a wrapper to evaluate zz from sdf functions on the given x,y plane defined by `xs` and `ys`


In [ ]:
def eval_sdf(xs, ys, sdFunc):
    zz = np.empty( (len(ys), len(xs)) )
    
    for j in range(len(ys)):
        for i in range(len(xs)):
            q = vec(xs[i],ys[j])
            zz[j,i] = sdFunc(q)
    return zz

## SDF explorer

In [ ]:
partial_sdfs = [sdUnitHline, sdUnitCircle, sdStar1, sdStar2]
# for f in partial_sdfs:
#     print (f)
#     functools.update_wrapper(f, f.func)
print(partial_sdfs)

In [ ]:
# functions that takes more than the query vector as arguments
generic_sdfs = [ sdLine, sdCircle, sdEquilateralTriangle, sdTriangle, sdStar]
nprint(partial_sdfs + generic_sdfs )

In [ ]:
# for f in partial_sdfs + generic_sdfs :
#     print(f)

In [ ]:
# sdfs to explore
sdfs = [ f for f in partial_sdfs + generic_sdfs ]
from bokeh.palettes import GnBu9
from collections import defaultdict
from functools import wraps

In [ ]:
# # Cache decorator for sdf's to be used in sdf explorer (ie. the key to the cache is the current sdf's parameter settings
# # Reference: https://is.gd/77xA20
# def memoize(orig_func):
#     memo = {}
#     @wraps(orig_func)
#     def wrapper(*args, **kwargs):
#         try:
# #             memo[...] # handling the key (as the class's properties is hard..)
#             result = orig_func(*args, **kwargs)
#             return result
    
#     return wrapper


In [ ]:
CACHE = {}
HITS = defaultdict(int)

In [ ]:
class sdfExplorer_v1(param.Parameterized):
    
    ################################################################################
    # Instance Parameters
    ################################################################################
    xrange = param.Range(default=(-2,2), bounds=(-10,10))
    yrange = param.Range(default=(-2,2), bounds=(-10,10))
    n_points = param.Integer(default=100, label='Number of points per axis')
    show_gradfield = param.Boolean(default=True, label='Show gradient field')
    sdf = param.Selector( objects=partial_sdfs, label='SDF')

    
    ################################################################################
    # Constant class properties
    ################################################################################
    H, W = 500,500
    img_opts = opts.Image(height=H, width=W, colorbar_position='bottom')
    vfield_opts = opts.VectorField(width=W, height=H, color='Magnitude',
                                   pivot='tip',
                                   rescale_lengths=True)
    contour_opts = opts.Contours(width=W, height=H, 
                                 colorbar=False, 
                                 cmap='gray',
                                 tools=['hover'])
    overlay_opts = opts.Overlay(width=W, height=H)
    
    
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.kernel = kwargs.get('kernel', np.array([[-0.5, 0, 0.5]]) )
        self.set_dmap_overlay()
    
    
    ################################################################################
    # Holoviews components, Parmaeter Dependencies
    ################################################################################
    @param.depends("xrange", "yrange", "n_points", "sdf", watch=True)
    def get_img(self):
        xs = np.linspace(*self.xrange,num=self.n_points)
        ys = np.linspace(*self.yrange,num=self.n_points)
        key = str(self.xrange, self.yrange, self.n_points, self.sdf)

        try:
            zz = CACHE[key]
            HITS[key] += 1

        except KeyError:
            zz = eval_sdf(xs, ys, self.sdf)
            CACHE[key] = zz
        
        zz_img = hv.Image( (xs, ys, zz) ) \
                    .opts(self.img_opts)\
                    .opts(xlim=self.xrange, ylim=self.yrange)
        zz_contour_op = lambda: hv.operation.contours(zz_img, levels=0) \
                    .opts(self.contour_opts) \
                    .opts(xlim=self.xrange, ylim=self.yrange)
        return zz_img * zz_contour

        # compute gradients
        gradx = correlate2d(zz, self.kernel, mode='same')
        grady = correlate2d(zz, self.kernel.T, mode='same')
        ang, mag = u.UV2angMag(gradx, grady)
        
        gradfield_op = lambda: hv.VectorField((xs, ys, ang, mag)).opts(self.vfield_opts)
        
        dmap_img = datashade(zz_img, cmap=GnBu9) * hv.DynamicMap(zz_contour_op)
        dmap_gradfield = hv.DynamicMap(gradfield_op)
        
        self.dmap_img = dmap_img
        self.dmap_gradfield = dmap_gradfield
        
        return dmap_img * dmap_gradfield
    
    ################################################################################
    # Display DynammicMaps
    ################################################################################ 
#     @param.depends("show_gradfield", watch=True)
    def viewable(self):
        return self.dmap_img

In [ ]:
class sdfExplorer(param.Parameterized):
    
    ################################################################################
    # Instance Parameters
    ################################################################################
    xrange = param.Range(default=(-2,2), bounds=(-10,10))
    yrange = param.Range(default=(-2,2), bounds=(-10,10))
    n_points = param.Integer(default=100, label='Number of points per axis')
    show_gradfield = param.Boolean(default=False, label='Show gradient field')
    show_as_img = param.Boolean(default=True, label='Show sdf as image')
    show_as_ds = param.Boolean(default=False, label='Show sdf as datashaded image')

    sdf = param.Selector( objects=partial_sdfs, label='SDF')

    
    ################################################################################
    # Constant class properties
    ################################################################################
    H, W = 500,500
    img_opts = opts.Image(height=H, width=W, colorbar_position='bottom')
    rgb_opts = opts.RGB(height=H, width=W)

    ds_opts = dict(height=H, width=W)

    vfield_opts = opts.VectorField(width=W, height=H, color='Magnitude',
                                   pivot='tip',
                                   rescale_lengths=True)
    contour_opts = opts.Contours(width=W, height=H, 
                                 colorbar=False, 
                                 cmap='gray',
                                 tools=['hover'])
    overlay_opts = opts.Overlay(width=W, height=H)
    
    
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.count = defaultdict(int)
        self.kernel = kwargs.get('kernel', np.array([[-0.5, 0, 0.5]]) )

        self.dmap_img = hv.DynamicMap(self.get_img)
        self.dmap_ds = datashade(hv.DynamicMap(self.get_img), **self.ds_opts).opts(self.rgb_opts)
        self.dmap_gradfield = hv.DynamicMap(self.get_gradfield)
        self.set_viewable()
    
    
    ################################################################################
    # Holoviews components, Parmaeter Dependencies
    ################################################################################
    
    @param.depends("xrange", "yrange", "n_points", "sdf", watch=True)
    def get_img(self):
        self.count['get_img'] += 1
        xs = np.linspace(*self.xrange,num=self.n_points)
        ys = np.linspace(*self.yrange,num=self.n_points)
        key = str((self.xrange, self.yrange, self.n_points, self.sdf))

        try:
            zz = CACHE[key]
            HITS[key] += 1

        except KeyError:
            zz = eval_sdf(xs, ys, self.sdf)
            CACHE[key] = zz
        
        zz_img = hv.Image( (xs, ys, zz) ).opts(self.img_opts) \
                    .opts(xlim=self.xrange, ylim=self.yrange)
        zz_contour = hv.operation.contours(zz_img, levels=0).opts(self.contour_opts) \
                    .opts(xlim=self.xrange, ylim=self.yrange)
        return zz_img * zz_contour
    
    @param.depends("xrange", "yrange", "n_points", "sdf", watch=True)
    def get_gradfield(self):
        self.count['get_gradfield'] += 1
        xs = np.linspace(*self.xrange,num=self.n_points)
        ys = np.linspace(*self.yrange,num=self.n_points)
        key = str((self.xrange, self.yrange, self.n_points, self.sdf))

        try:
            zz = CACHE[key]
            HITS[key] += 1

        except KeyError:
            zz = eval_sdf(xs, ys, self.sdf)
            CACHE[key] = zz
            
        # compute gradients
        gradx = correlate2d(zz, self.kernel, mode='same')
        grady = correlate2d(zz, self.kernel.T, mode='same')
        ang, mag = u.UV2angMag(gradx, grady)
        
        gradfield = hv.VectorField((xs, ys, ang, mag)).opts(self.vfield_opts)
        return gradfield
    
    @param.depends('show_gradfield', 'show_as_img', 'show_as_ds', watch=True)
    def set_viewable(self):
        self.count['set_viewable'] += 1
        overlay = hv.Overlay()
        
        if self.show_as_img:
            overlay *= self.dmap_img
        if self.show_as_ds:
            overlay *= self.dmap_ds
        if self.show_gradfield:
            overlay *= self.dmap_gradfield

        self.viewable = overlay
    
    ################################################################################
    # Display DynammicMaps
    ################################################################################ 
    @param.depends('show_gradfield', 'show_as_img', 'show_as_ds', watch=True)
    def view(self):
        return self.viewable
        

In [ ]:
ex = sdfExplorer()

In [ ]:
pn.Column(
    pn.Param(ex.param), 
    pn.panel(ex.view)
)

In [ ]:
functools.update_wrapper(temp, temp.func)#s.func)

In [ ]:
ex._set_dmap()

In [ ]:
    integer_range           = param.Range(default=(3,7),bounds=(0, 10))
